# 使用LCEL语法构建一个简单链

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [18]:
import os

llm = ChatOpenAI(
    model = "DMXAPI-DeepSeek-V3-Fast",
    api_key = os.getenv("DMX_API_KEY"),
    base_url = os.getenv("DMX_BASE_URL")
)

In [19]:
prompt = ChatPromptTemplate.from_template("使用简短的句子总结{topic}")

In [20]:
parser = StrOutputParser()

In [21]:
chain = prompt | llm | parser

# 实时流式输出

这个函数通过 chain.stream 方法为指定主题生成数据流，并遍历这些数据，立即输出每条数据的 content 。使用 end="" 参数可以禁止输出后添加换行符，而 flush=True 参数则会使输出缓冲区立即清空。

In [22]:
for chunk in chain.stream({"topic" : "人等于动物，有人做牛马，天天吃苦，否则吃不到饭。有人做猫做蝴蝶，一辈子好吃懒做，东张西望，照样享福。"}):
    print(chunk,end = "",flush = True)

人生如畜，劳逸天定：牛马苦耕方得食，猫蝶闲晃亦享福。

# 调用

In [23]:
chain.invoke({"topic" : "那一天，我也将沉静着走下山去，扶着我的拐杖。有一天，在某一处山洼里，势必会跑上来一个欢蹦的孩子，抱着他的玩具。 当然，那不是我。 但是，那不是我吗？ 宇宙以其不息的欲望将一个歌舞炼为永恒。这欲望有怎样一个人间的姓名，大可忽略不计。"})

'**生命如歌，代代更迭。**  \n我拄杖下山，静默如尘；  \n山洼里跃出孩童，怀抱天真。  \n“非我”亦是“我”——  \n宇宙以不息之欲，将刹那舞成永恒。  \n姓名皆虚妄，唯有生生不息。  \n\n（注：以简练短句呼应原文哲思，保留“逝去-新生”的轮回意象，末句点破“无名永恒”的宇宙本质。）'

# 批处理：单元执行

函数 chain.batch 接受一个包含多个字典的列表作为参数，并利用每个字典中的 topic 键的值执行批量处理。

In [24]:
chain.batch([{"topic" : "不能承受生命之痛，残疾的躯体，在轮椅上思考，从大自然中得到启示，死是一个必然降临的节日，应该健康的活着，恪守并遵循生命的轨迹。"},
            {"topic" : "我在这园子里坐着，园神成年累月地对我说：孩子，这不是别的，这是你的罪孽和福祉。"}])

['残疾之躯，轮椅沉思。  \n\n自然启示，生死顿悟。  \n\n死如节日，终将赴约。  \n\n生当健康，顺命而行。',
 '“我”在园中静坐，园神长年告诫：“这是你的罪孽与福祉。”']

配置字典通过max_concurrency键来设置可并发处理的操作的最大数量

In [25]:
chain.batch([
    {"topic" : "太阳，他是夕阳，太阳正在升起。当他走出下山收集这些荒凉的照片时，正是他燃烧另一面并爬上山顶布散和朝辉时。那一天，我要走下山，扶着我的拐杖。在某个地方山洼，会有一个快乐的孩子抱着他的玩具。当然，那不是我。那不是我？宇宙以其无尽的欲望，使一支歌舞永恒。这个愿望有什么名字？可以忽略不计。"},
    {"topic" : "当一个人出生时，这不再是一个可以争论的问题，而是上帝赋予他的事实，当上帝给我们这个事实时，他已经保证了他的结果，因此死亡是一件不必急于成功的事，而死亡是一个必然会到来的节日。"},
    {"topic" : "有些东西只适合收藏。我们不能说，我们不能想，但我们不能忘记……，他们是朦胧的温馨和孤独，他们是成熟的希望和绝望，他们只有两个领地：心灵和坟墓。"}
],
config = {"max_concurrency" : 3})

['太阳沉落又新生，燃烧轮回。我拄杖下山，山洼里嬉戏的孩子不是我——或真是我？宇宙以无名之欲，将刹那舞姿凝为永恒。  \n\n（注：用短句拆解原诗意象，保留"夕阳/朝阳"的循环、"孩子与自我"的镜像疑问、"宇宙欲望"的哲思，末句以"刹那与永恒"的矛盾收束，呼应"忽略不计"的虚无感。）',
 '1. 出生已成定局，是神赐的事实。  \n2. 神给予生命时，已注定结局。  \n3. 死亡无需追逐，终会降临。  \n4. 死亡是必然的节日。',
 '有些东西只能珍藏。无法言说，无法思考，却无法遗忘。它们是模糊的温暖与孤寂，是成熟的期盼与幻灭。它们仅存于两处：心底或坟墓。']

# 异步数据流

函数 chain.astream 会创建一个异步流，用于异步处理指定主题的消息。

它通过异步循环（ async for ）依次从流中接收消息，并利用打印函数即时输出消息内容（ s.content ）。 end="" 设置禁止打印后自动换行， flush=True 强制清空输出缓冲区，保证消息能立即显示。

In [26]:
async for chunk in chain.astream({"topic" : "太阳，他是夕阳，太阳正在升起。当他走出下山收集这些荒凉的照片时，正是他燃烧另一面并爬上山顶布散和朝辉时。那一天，我要走下山，扶着我的拐杖。在某个地方山洼，会有一个快乐的孩子抱着他的玩具。当然，那不是我。那不是我？宇宙以其无尽的欲望，使一支歌舞永恒。这个愿望有什么名字？可以忽略不计。"}):
    print(chunk,end = "",flush = True)

太阳是轮回的诗：  
夕阳沉没，晨光重生。  
他下山拾荒凉，转身点燃山巅。  

我拄杖离去，  
山洼里嬉戏的孩子——  
“不是我”的回声。  

宇宙的舞永不落幕，  
无名之欲，  
一粒尘埃的重量。

# 异步执行

第一个方法（对象）会使用给定的参数异步执行操作

In [27]:
process = chain.ainvoke({"topic" : "DeepSeek"})

await process

'DeepSeek 是深度求索公司开发的 AI 助手，擅长文本理解与生成，支持长文本（128K）、文件处理，免费高效。'

# 异步批处理

该函数 abatch 异步批量处理一系列动作。

在这个示例中，我们使用 abatch 对象的 chain 方法来异步处理 topic 上的动作。

" await 关键字用于等待所有异步任务完成。"

In [28]:
process_batch = chain.abatch([
    {"topic" : "月之暗面"},
    {"topic" : "通义千问"},
    {"topic" : "豆包"}
])

await process_batch

['月之暗面指月球永远背对地球的一面。  \n特点：  \n1. 无法从地球直接观测  \n2. 地形更崎岖，陨石坑更多  \n3. 首次由苏联探测器拍摄（1959年）  \n4. 中国嫦娥四号首次软着陆（2019年）  \n5. 与正面地质差异显著  \n\n注：并非"永远黑暗"，也有昼夜交替。',
 '通义千问是阿里云推出的AI大模型，支持多轮对话、多语言及多模态交互，具备知识问答、内容生成、逻辑推理等能力，适用于办公、学习、娱乐等场景。特点包括：高效精准、持续进化、开放API，赋能企业及开发者。',
 '豆包是字节跳动推出的AI助手，功能类似ChatGPT，支持文本生成、问答、翻译等。特点：免费、中文优化、多端可用（APP/网页）。']

# parallel

使用 langchain_core.runnables 模块中的 RunnableParallel 类并行运行两个任务的示例。

创建两个链 (chain1, chain2)，使用 ChatPromptTemplate.from_template 方法获取给定 country 的首都和面积。

这些链通过 model 和管道(|)运算符分别连接。最后，我们使用 RunnableParallel 类，以 capital 和 area 为键组合这两个链，创建一个可并行运行的 combined 对象。

In [29]:
from langchain_core.runnables import RunnableParallel


chain_1 = ChatPromptTemplate.from_template("告诉我{country}首都是哪个？") | llm | parser

chain_2 = ChatPromptTemplate.from_template("告诉我{country}的国土面积。") | llm | parser

parallel = RunnableParallel(capital = chain_1,area = chain_2)

In [30]:
parallel.invoke({"country" : "中国"})

{'capital': '中国的首都是**北京**。北京是中国的政治、文化、国际交往和科技创新中心，拥有悠久的历史和丰富的文化遗产，如故宫、天安门广场和长城等标志性景点。',
 'area': '中国的国土面积约为 **960万平方公里**，是世界第三大国家（仅次于俄罗斯和加拿大）。这一数据通常指的是陆地面积，包括中国大陆、台湾地区（尽管政治地位存在争议）、香港和澳门特别行政区。  \n\n如果包括领海、专属经济区等水域，中国的总面积约为 **1,040万平方公里**。  \n\n需要注意的是，不同来源可能因统计标准（如是否计入争议领土、水域等）略有差异，但官方主流数据以 **960万平方公里** 为准。'}

In [31]:
parallel.batch([
    {"country" : "美国"},{"country" : "德国"}
])

[{'capital': '美国的首都是**华盛顿哥伦比亚特区**（Washington, D.C.），通常简称为**华盛顿**或**D.C.**。它位于美国的东海岸，是一个独立的联邦行政区，不属于任何州。  \n\n**关键点：**  \n- **名称来源**：以美国首任总统乔治·华盛顿命名，"哥伦比亚"是美国的诗意别称。  \n- **重要地标**：白宫、国会大厦、林肯纪念堂等均坐落于此。  \n- **常见误解**：华盛顿州（位于西海岸，首府为奥林匹亚）与华盛顿D.C.是不同的地方。  \n\n如果有其他相关问题，欢迎继续提问！ 😊',
  'area': '美国的国土面积根据不同的计算方式有所差异，以下是主要数据：\n\n1. **陆地面积**：约9,147,593平方公里（3,531,905平方英里）  \n2. **水域面积**：包括内陆水域和五大湖等，约664,709平方公里（256,645平方英里）  \n3. **总面积（陆地+水域）**：约9,833,517平方公里（3,796,742平方英里）  \n\n**注意事项**：  \n- 若仅计陆地面积，美国是世界第三大国（仅次于俄罗斯和中国）；若包含水域，则排名第四（加上加拿大）。  \n- 数据来源可能因测量标准（如海岸线算法、领土争议）略有差异，例如《世界概况》（CIA Factbook）和美国人口普查局的数据可能微小差别。  \n\n如需更精确用途，建议参考官方机构的最新统计。'},
 {'capital': '德国的首都是**柏林**（Berlin）。柏林不仅是德国最大的城市，也是其政治、文化、教育和交通中心。自1990年德国统一后，柏林取代了之前的波恩（Bonn）成为首都。\n\n**小知识**：  \n- 柏林以丰富的历史、艺术氛围和活跃的夜生活闻名。  \n- 勃兰登堡门（Brandenburg Gate）和柏林墙遗址是标志性景点。  \n- 尽管波恩在东西德分裂时期曾是西德的“临时首都”，但统一后政府机构逐步迁至柏林。  \n\n如果有其他关于德国或柏林的问题，欢迎继续提问！ 🌍',
  'area': '德国的国土面积约为 **357,592 平方公里**（根据最新数据，包括内陆水域）。这一数字可能因统计标准（如是否计入水域面积）略有差异，但通常采用联合国或德国官方公布的约35.

In [32]:
for chunk in parallel.stream({"country" : "中国"}):
    print(chunk,end = "",flush = True)

{'area': ''}{'area': '中国的'}{'area': '国土'}{'area': '面积约为 **'}{'area': '960万平方公里**'}{'area': '，是世界'}{'area': '第三'}{'area': '大国家'}{'capital': ''}{'capital': '中国的'}{'area': '（仅次于俄罗斯和'}{'capital': '首都是**北京'}{'capital': '**。北京不仅是'}{'area': '加拿大）。这一'}{'capital': '中国的政治、'}{'capital': '文化、'}{'area': '数据通常'}{'capital': '教育'}{'capital': '中心，也是历史悠久'}{'area': '包括陆地面积和'}{'capital': '的世界著名'}{'area': '内陆'}{'capital': '古都之一'}{'capital': '，拥有故宫'}{'area': '水域，'}{'capital': '、天'}{'capital': '安门、'}{'area': '如'}{'capital': '长城等众多'}{'capital': '标志性景观'}{'area': '湖泊'}{'capital': '。  \n\n如果有'}{'area': '、河流等'}{'capital': '其他关于'}{'capital': '中国'}{'capital': '或北京的问题，'}{'area': '。  \n\n此外'}{'capital': '欢迎随时提问！'}{'capital': ' 😊'}{'area': '，中国还拥有'}{'area': '广阔的'}{'area': '海洋领土'}{'area': '，包括'}{'area': ' **'}{'area': '约300万平方公里'}{'area': '** '}{'area': '的管辖海域（'}{'area': '根据《联合国'}{'area': '海洋法公约'}{'area': '》主张'}{'area': '的专属经济区和'}{'area': '大陆架）。因此'}{'area': '，中国的陆'}{'area': '海总面积'}{'area': '约为 **1'}{'area': ',260'}{'ar